In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.express as px
import os

In [3]:
df = pd.read_csv("/kaggle/input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [7]:
df.columns = (
    df.columns
      .str.replace(".", "_", regex=False)  # noktaları _ ile değiştir
      .str.replace(" ", "_")               # boşlukları _ ile değiştir
      .str.upper()                         # büyük harfe çevir
)

In [11]:
conn = sql.connect("/kaggle/working/credit_dwh.db")
df.to_sql( name="RAW_CREDIT", con=conn, index=False, if_exists="replace" )

30000

In [13]:
conn.execute(""" CREATE TABLE DIM_CUSTOMER AS SELECT DISTINCT ID AS CLIENT_ID, SEX, EDUCATION, MARRIAGE, AGE FROM RAW_CREDIT """)

In [14]:
conn.execute(""" CREATE TABLE DIM_RISK AS SELECT DISTINCT ID AS CLIENT_ID, CASE WHEN DEFAULT_PAYMENT_NEXT_MONTH = 1 THEN 'High Risk' WHEN PAY_0 > 1 THEN 'Medium Risk' ELSE 'Low Risk' END AS RISK_SEGMENT FROM RAW_CREDIT """)

In [15]:
pd.read_sql("""
SELECT name
FROM sqlite_master
WHERE type='table'
""", conn)


,name
0,RAW_CREDIT
1,DIM_CUSTOMER
2,DIM_RISK


In [16]:
df_dim_customer = pd.read_sql(
    "SELECT * FROM DIM_CUSTOMER LIMIT 10",
    conn
)
df_dim_customer


,CLIENT_ID,SEX,EDUCATION,MARRIAGE,AGE
0,1,2,2,1,24
1,2,2,2,2,26
2,3,2,2,2,34
3,4,2,2,1,37
4,5,1,2,1,57
5,6,1,1,2,37
6,7,1,1,2,29
7,8,2,2,2,23
8,9,2,3,1,28
9,10,1,3,2,35


In [17]:
df_dim_risk = pd.read_sql(
    "SELECT * FROM DIM_RISK LIMIT 10",
    conn
)
df_dim_risk


,CLIENT_ID,RISK_SEGMENT
0,1,High Risk
1,2,High Risk
2,3,Low Risk
3,4,Low Risk
4,5,Low Risk
5,6,Low Risk
6,7,Low Risk
7,8,Low Risk
8,9,Low Risk
9,10,Low Risk


In [18]:
conn.execute("DROP TABLE IF EXISTS FACT_CREDIT")
conn.execute("""
CREATE TABLE FACT_CREDIT AS
SELECT
    ID AS CLIENT_ID,
    LIMIT_BAL,
    PAY_0,
    PAY_2, PAY_3, PAY_4, PAY_5, PAY_6,
    BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6,
    PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6,
    DEFAULT_PAYMENT_NEXT_MONTH AS DEFAULT_FLAG
FROM RAW_CREDIT
""")
conn.commit()


In [19]:
pd.read_sql("SELECT * FROM FACT_CREDIT LIMIT 10", conn)


,CLIENT_ID,LIMIT_BAL,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT_FLAG
0,1,20000.0,2,2,-1,-1,-2,-2,3913.0,3102.0,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,-1,2,0,0,0,2,2682.0,1725.0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,0,0,0,0,0,0,29239.0,14027.0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,0,0,0,0,0,0,46990.0,48233.0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,-1,0,-1,0,0,0,8617.0,5670.0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
5,6,50000.0,0,0,0,0,0,0,64400.0,57069.0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0
6,7,500000.0,0,0,0,0,0,0,367965.0,412023.0,...,542653.0,483003.0,473944.0,55000.0,40000.0,38000.0,20239.0,13750.0,13770.0,0
7,8,100000.0,0,-1,-1,0,0,-1,11876.0,380.0,...,221.0,-159.0,567.0,380.0,601.0,0.0,581.0,1687.0,1542.0,0
8,9,140000.0,0,0,2,0,0,0,11285.0,14096.0,...,12211.0,11793.0,3719.0,3329.0,0.0,432.0,1000.0,1000.0,1000.0,0
9,10,20000.0,-2,-2,-2,-2,-1,-1,0.0,0.0,...,0.0,13007.0,13912.0,0.0,0.0,0.0,13007.0,1122.0,0.0,0


In [20]:
df_kpi1 = pd.read_sql("""
SELECT
    R.RISK_SEGMENT,
    COUNT(*) AS CLIENT_COUNT
FROM DIM_RISK R
GROUP BY R.RISK_SEGMENT
""", conn)

df_kpi1


,RISK_SEGMENT,CLIENT_COUNT
0,High Risk,6636
1,Low Risk,22411
2,Medium Risk,953


In [21]:
df_kpi2 = pd.read_sql("""
SELECT
    R.RISK_SEGMENT,
    COUNT(*) AS TOTAL_CLIENTS,
    SUM(F.DEFAULT_FLAG) AS DEFAULT_COUNT,
    ROUND(1.0 * SUM(F.DEFAULT_FLAG) / COUNT(*), 3) AS DEFAULT_RATE
FROM FACT_CREDIT F
JOIN DIM_RISK R ON F.CLIENT_ID = R.CLIENT_ID
GROUP BY R.RISK_SEGMENT
""", conn)

df_kpi2


,RISK_SEGMENT,TOTAL_CLIENTS,DEFAULT_COUNT,DEFAULT_RATE
0,High Risk,6636,6636,1.0
1,Low Risk,22411,0,0.0
2,Medium Risk,953,0,0.0


In [22]:
df_kpi3 = pd.read_sql("""
SELECT
    R.RISK_SEGMENT,
    ROUND(AVG(F.LIMIT_BAL), 0) AS AVG_LIMIT
FROM FACT_CREDIT F
JOIN DIM_RISK R ON F.CLIENT_ID = R.CLIENT_ID
GROUP BY R.RISK_SEGMENT
""", conn)

df_kpi3


,RISK_SEGMENT,AVG_LIMIT
0,High Risk,130110.0
1,Low Risk,181099.0
2,Medium Risk,107572.0
